In [1]:
from environment_manager import *

Create the version Environment

In [2]:

name = "240308-initial"
env = load_create_environment(name)


Setting environment variables
Copying vanilla components
File copied successfully from C:\Users\jossi\Dropbox\Office_Work\Jos\GH_Graph_Learning\Grasshopper Components\240307-CoreComponents\vanilla_components.csv to ExtractionEnvironments\240308-initial\00-VanillaComponents\vanilla_components.csv.
Copying components
Copying gh files


In [3]:
# gh_file = env.dirs["files"] /"210309_Canopy Modelling.gh"
# # gh_file = env.dirs["files"] /'test.gh'
# doc = GHProcessor.get_ghdoc(str(gh_file))
# t_obj = ["Panel"]
# illegal = ["Sketch","Scribble"]
# test_objects = []
# canvas = Canvas(gh_file.stem, doc, env)
# for object in doc.Objects:
#     c = GHComponent(object)
#     print(f"{c.name}, {c.get_connections(canvas)}")


In [4]:
import Grasshopper.Kernel as ghk

In [ ]:
from Grasshopper.Kernel import GH_DocumentIO
file = r"C:\Users\jossi\Dropbox\Office_Work\Jos\GH_Graph_Learning\test\all_components.gh"
doc = GHProcessor.get_ghdoc(file)

comp = GHComponentTable
comp.initialise()

object_proxies = GHComponentTable.object_proxies


i_space = 100.
j_space = 200.
currenttype = object_proxies[0].category
pivot = [0, 0]
problems = [] 
for obj in object_proxies[:1000]:
    print(obj.name)
    # if obj.name == "FeatureRequest":
    #     print(obj)
    #     continue
    # if obj.category != currenttype:
    #     currenttype = obj.category
    #     pivot[1] += j_space
    # try:
    #     print(obj.name)
    #     dobj = obj.obj_proxy.CreateInstance()
    #     dobj.Attributes.Pivot = System.Drawing.PointF(pivot[0], pivot[1])
    #     doc.AddObject(dobj, False)
    #     pass
    # except AttributeError as e:
    #     problems.append(obj)
    #     print(f"{e}:{obj.name}")
    #     continue
    # except Exception as e:
    #     problems.append(obj)
    #     print(f"{e}:{obj.name}")
    # pivot[0] += i_space
# ghdio = GH_DocumentIO(doc)
# ghdio.Save()
# print(problems)